# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull.

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it.

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link.

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell.

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.*


# Importing Libraries

In [27]:
import os
import datetime
import re

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

In [10]:
HEADERS = {'User-Agent': 'Mozilla/5.0'}

---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways.

In [28]:
artists = {
    'billie_eilish': 'https://www.azlyrics.com/b/billieeilish.html',
    'adele':         'https://www.azlyrics.com/a/adele.html'
}

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).)

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again.

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages.

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know?

A: <!-- Delete this comment and put your answer here. -->


In [29]:
lyrics_pages = defaultdict(list)


for artist, artist_url in artists.items():
    # Fetch the artist page
    resp = requests.get(artist_url, headers=HEADERS)
    resp.raise_for_status()

    # Parse the page HTML
    soup = BeautifulSoup(resp.text, 'html.parser')

    # Find every <div class="listalbum-item"> (each holds one song link)
    song_divs = soup.find_all('div', class_='listalbum-item')

    # Pull the <a> inside each div, build the full URL, and store it
    for div in song_divs:
        a_tag = div.find('a')                                # the <a href="../lyrics/...">
        rel_link = a_tag['href']                             # e.g. "../lyrics/billieeilish/bad-guy.html"
        full_url = 'https://www.azlyrics.com' + rel_link.replace('..', '')
        lyrics_pages[artist].append(full_url)

    # Report how many songs we found
    print(f"{artist}: found {len(lyrics_pages[artist])} songs")

billie_eilish: found 80 songs
adele: found 71 songs


Let's make sure we have enough lyrics pages to scrape.

In [13]:
for artist, urls in lyrics_pages.items():
    assert len(urls) >= 20, f"Only {len(urls)} URLs found for {artist}"

In [14]:
# Let's see how long it's going to take to pull these lyrics
# if we're waiting `5 + 10*random.random()` seconds
for artist, urls in lyrics_pages.items():
    print(f"For {artist} we have {len(urls)}.")
    print(f"The full pull will take for this artist will take {round(len(urls)*10/3600,2)} hours.")

For billie_eilish we have 80.
The full pull will take for this artist will take 0.22 hours.
For adele we have 71.
The full pull will take for this artist will take 0.2 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part.

1. Create an empty folder in our repo called "lyrics".
1. Iterate over the artists in `lyrics_pages`.
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages.
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name.


In [15]:
def generate_filename_from_link(link) :

    if not link :
        return None

    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")

    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")

    # tack on .txt
    name = name + ".txt"

    return(name)


In [16]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics") :
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

In [25]:
url_stub = "https://www.azlyrics.com"
start = time.time()

total_pages = 0

for artist, urls in lyrics_pages.items():
    # Create a subfolder for this artist
    artist_dir = os.path.join("lyrics", artist)
    os.makedirs(artist_dir, exist_ok=True)

    for link in urls[:20]:


        # Fetch the page (polite pause before each request)
        time.sleep(5 + 10 * random.random())
        resp = requests.get(link, headers=HEADERS)
        resp.raise_for_status()

        # Parse out the lyrics
        page_soup = BeautifulSoup(resp.text, 'html.parser')
        lyrics_div = page_soup.find_all('div', attrs={'class': None, 'id': None})[0]
        lyrics_text = lyrics_div.get_text(separator='\n').strip()

        # Build filename, then write title + two newlines + lyrics
        filename = generate_filename_from_link(link)
        out_path = os.path.join(artist_dir, filename)
        with open(out_path, 'w', encoding='utf-8') as f:
            # title is the first line of the lyrics block (optional)
            title = lyrics_text.split('\n', 1)[0]
            f.write(title + "\n\n" + lyrics_text)

    # Report how many songs we saved for this artist
    saved = len(os.listdir(artist_dir))
    print(f"{artist}: saved {saved} lyrics files")

billie_eilish: saved 55 lyrics files
adele: saved 20 lyrics files


In [31]:
elapsed_hours = round((time.time() - start) / 3600, 2)
print(f"Total run time: {elapsed_hours} hours across {total_pages} pages.")

Total run time: 0.38 hours across 0 pages.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [32]:
def words(text):
    """Split text into lowercase words using regex."""
    return re.findall(r'\w+', text.lower())

## Checking Lyrics

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work.

In [34]:
artists = ['billie_eilish', 'adele']
for artist in artists:
    # Build the path to the artist’s lyrics folder
    artist_dir = os.path.join('lyrics', artist)

    # List only .txt files in that folder
    artist_files = [
        fname for fname in os.listdir(artist_dir)
        if fname.lower().endswith('.txt')
    ]

    # Report how many lyric files we have
    num_files = len(artist_files)
    print(f"For {artist} we have {num_files} files.")

    # Read each file, extract words, and accumulate
    all_words = []
    for fname in artist_files:
        file_path = os.path.join(artist_dir, fname)
        with open(file_path, encoding='utf-8') as infile:
            text = infile.read()
        all_words.extend(words(text))

    # Compute total and unique word counts
    total_words  = len(all_words)
    unique_words = len(set(all_words))
    print(f"For {artist} we have roughly {total_words} words, {unique_words} are unique.\n")


For billie_eilish we have 55 files.
For billie_eilish we have roughly 14245 words, 840 are unique.

For adele we have 20 files.
For adele we have roughly 5977 words, 778 are unique.

